## Рынок заведений общественного питания Москвы

Описание проекта  

Мы решили открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Вместе с партнёрами мы решились обратиться к инвесторам. Их интересует текущее положение дел на рынке — сможем ли мы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов?  

У нас есть открытые данные о заведениях общественного питания в Москве.

In [43]:
import pandas as pd
rest_data = pd.read_csv('datasets/rest_data.csv')

In [44]:
rest_data.head()

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
0,0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48,улица Егора Абакумова
1,1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35,улица Талалихина
2,2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95,Абельмановская улица
3,3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40,Абрамцевская улица
4,4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50,Абрамцевская улица


Таблица rest_data:
* object_name — название объекта общественного питания;
* chain — сетевой ресторан;
* object_type — тип объекта общественного питания;
* address — адрес;
* number — количество посадочных мест.

In [45]:
# Проверим типы данных в столбцах и отсутствие пропусков
rest_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15366 entries, 0 to 15365
Data columns (total 8 columns):
Unnamed: 0     15366 non-null int64
id             15366 non-null int64
object_name    15366 non-null object
chain          15366 non-null object
object_type    15366 non-null object
address        15366 non-null object
number         15366 non-null int64
street         15366 non-null object
dtypes: int64(3), object(5)
memory usage: 960.5+ KB


Типы данных соотвествуют столбцам, пропусков в данных нет.

In [46]:
# Проверим данные на дубликаты
rest_data.duplicated().sum()

0

<font style="color:DeepPink; font-size:400%">•</font> **Анализ данных**

<font style="color:DeepPink; font-size:200%">•</font> Исследуем соотношение типов объектов общественного питания по количеству.

In [47]:
# Сгруппируем данные по типу объектов.
type_data = (rest_data
             .groupby('object_type')
             .agg({'id': 'count'})
             .sort_values(by='id', ascending = False)
             .reset_index()
            )
# Добавим идентификатор, общим данным по сетевым и не сетевым заведениям.
type_data['chain'] = 'все'
# Сгруппируем данные по виду сетевой-не сетевой и типу объектов.
group_type_chain_data = rest_data.groupby(['chain', 'object_type']).agg({'id':'count'}).reset_index()
# Добавим понятные идентификаторы
group_type_chain_data['chain'] = group_type_chain_data['chain'].replace('да', 'сетевые')
group_type_chain_data['chain'] = group_type_chain_data['chain'].replace('нет', 'не сетевые')
# Объединим полученные таблицы
type_data = type_data.append(group_type_chain_data, sort=True)

In [48]:
import plotly.express as px
fig = px.bar(type_data, y='id', x='object_type', color='chain', barmode='group', 
             labels={'id': 'Количество бъектов', 'object_type': ''}, 
             title='Cоотношение типов объектов общественного питания по количеству',
             template='plotly_white', color_discrete_sequence=px.colors.qualitative.Plotly[-3::-2])
fig.update_xaxes(tickangle=45)
fig.show()

Больше других типов заведений в Москве распространены:
* кафе - 6099.
* столовые - 2587
* рестораны - 2285

<font style="color:DeepPink; font-size:200%">•</font> Исследуем соотношение сетевых и несетевых заведений по количеству.

In [49]:
# Сгруппируем данные по столбцу chain и посчитаем количество заведений в каждой группе
chain_col = (rest_data
             .groupby('chain')
             .agg({'id': 'count'})
             .sort_values(by='id', ascending = False)
             .reset_index()
            )
# Добавим столбец с отношением между группами в процентах
chain_col['%'] = round(chain_col['id'] / chain_col['id'].sum() *100)
chain_col

,chain,id,%
0,нет,12398,81.0
1,да,2968,19.0


19% рынка общественного питания составляют сетевые заведения.

<font style="color:DeepPink; font-size:200%">•</font> Определим для какого вида объекта общественного питания характерно сетевое распространение.

In [50]:
fig.show()

 Основную массу сетевых заведений составляют: кафе, предприятия быстрого обслуживания и рестораны.  
 Впрочем, как и среди не сетевых заведений самый распространенный формат: кафе.

<font style="color:DeepPink; font-size:200%">•</font> Определим что характерно для сетевых заведений:  
* много заведений с небольшим числом посадочных мест в каждом или 
* мало заведений с большим количеством посадочных мест?

In [51]:
# Отфильтруем сетевые заведения в отдельную таблицу
chain_data = rest_data.query('chain == "да"').copy()
chain_data.head()

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
3,3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40,Абрамцевская улица
6,6,148815,Брусника,да,кафе,"город Москва, переулок Сивцев Вражек, дом 6/2",10,переулок Сивцев Вражек
13,13,22579,Алло Пицца,да,кафе,"город Москва, улица Авиаторов, дом 14",32,улица Авиаторов
16,16,144107,Суши Wok,да,предприятие быстрого обслуживания,"город Москва, Азовская улица, дом 3",7,Азовская улица
18,18,58565,Тануки,да,ресторан,"город Москва, Большая Академическая улица, дом 65",160,Большая Академическая улица


In [52]:
# В названиях сетей попадаются дубликаты (например, Макдоналдс и МАКДОНАЛДС, 
# Шоколадница, ШОКОЛАДНИЦА и Кафе «Шоколадница». Заменим некорректные названия в крупных сетях вручную.
replace_map = ({'МАКДОНАЛДС': 'Макдоналдс', 'ШОКОЛАДНИЦА': 'Шоколадница', 'Кафе «Шоколадница»': 'Шоколадница', 
                'Старбакс': 'Starbucks', 'БУРГЕР КИНГ': 'Бургер Кинг', 'Сабвей': 'Subway', 'Кафе «KFC»': 'KFC', 
                'Суши Вок': 'Суши Wok', 'ТЕРЕМОК': 'Теремок', 'Братья Караваевы': 'Кулинарная лавка братьев Караваевых', 
                'Ресторан «Макдоналдс»': 'Макдоналдс', 'Хлеб Насущный': 'Хлеб насущный', 'МИЛТИ': 'Милти', 
                'Бургер Кинг Burger King': 'Бургер Кинг', 'Ростикс KFC': 'KFC', 'Пиццерия «Папа Джонс»': 'Папа Джонс', 
                'Кафе «Теремок»': 'Теремок', 'Андерсон': 'АндерСон', 'Прайм-кафе': 'Прайм', 
                'Крошка картошка': 'Крошка Картошка', 'Предприятие быстрого обслуживания «Макдоналдс»': 'Макдоналдс', 
                'ТАНУКИ': 'Тануки', 'Ресторан «Тануки»': 'Тануки', 'МУ-МУ': 'Му-Му', 'Прайм стар': 'Прайм', 
                'Додо пицца': 'Додо Пицца'})
chain_data['object_name'] = chain_data['object_name'].map(replace_map).fillna(chain_data['object_name'])

In [53]:
# Определим ТОП10 сетевых заведений по количеству точек, расчитаем медиану количества мест
chain_name_data = (chain_data.groupby('object_name')
             .agg({'id': 'count', 'number': 'median'})
             .sort_values(by='id', ascending=False)
             .reset_index()
            )
top10_chain_name_data = chain_name_data.head(10)
top10_chain_name_data

,object_name,id,number
0,KFC,173,50.0
1,Шоколадница,171,50.0
2,Макдоналдс,165,79.0
3,Бургер Кинг,140,45.0
4,Теремок,101,24.0
5,Крошка Картошка,89,15.0
6,Домино'с Пицца,86,16.0
7,Милти,72,0.0
8,Суши Wok,71,6.0
9,Папа Джонс,57,20.0


Вот наш топ-10 сетевых заведений по количеству точек.  
Посмотрим на распределение значений по количеству посадочных мест в 10-ке.

In [54]:
# Сохраним названия заведений из ТОП10 в список
top10_chain_names = top10_chain_name_data['object_name'].unique()

# Отфильтруем таблицу с сетевыми заведениями по названиям из списка ТОП10
filt_df = chain_data['object_name'].isin(top10_chain_names)
filt_chain_data_top10 = chain_data[filt_df]

# Отдельно добавим столбик с общим количеством заведений в сети и отсортируем в порядке убывания количества точек
quantity_of_chain = top10_chain_name_data[['object_name','id']]
quantity_of_chain.columns = ['object_name', 'quantity_of_chain']
filt_chain_data_top10 = filt_chain_data_top10.merge(quantity_of_chain, on='object_name').sort_values(by='quantity_of_chain')

In [55]:
fig = px.box(filt_chain_data_top10, x="number", y="object_name", orientation='h', 
             labels={'number': 'число посадочных мест', 'object_name': ''})
fig.show()

В KFC есть точки от 0 до 400 посадочных мест (медиана - 50).  
Широкий разброс значений и у Макдоналдса до 580 посадочных мест, при медианном значении - 79 мест.  
В сетях по-меньше разброс значений уже.  

Исходя из графика выше можно сделать вывод, что в крупных сетях среднее количество мест в заведении не превышает 79.

Посмотрим на распределение значений количества заведений в сети и количества посадочных мест в первых 50 крупных сетях.

In [56]:
fig = px.scatter(chain_name_data.head(50), y="number", x='id', color='object_name', size='id',
                 labels={'number': 'число посадочных мест', 'id': 'количество заведений'},
                 template='plotly_white', hover_name='object_name', 
                 color_discrete_sequence=px.colors.qualitative.Light24)
fig.update_layout(showlegend=False)
fig.show()

* В небольших сетях (до 20 заведений) наблюдается широкий диапазон значений числа посадочных мест: от 0 до 110.
* Сети по-крупнее (до 60 заведений) ограничиваются в основном 50 посадочными местами.
* Только Иль Патио, Чайхона, Тануки и Якитория в этой группе выделились в отдельную группу со 100 посадочными местами.
* И сеть "кафе-столовых" Му-Му насчитывает 23 точки в среднем со 150 посадочными местами.  
* Более крупные сети наращивают объемы пропорционально количеству заведений: чем крупнее сеть, тем больше она может обслужить посетителей.

<font style="color:DeepPink; font-size:200%">•</font> Для каждого вида объекта общественного питания определим среднее количество посадочных мест.  
Выясним, какой вид предоставляет в среднем самое большое количество посадочных мест?

In [57]:
type_number_data = (rest_data
                    .groupby('object_type')
                    .agg({'number': 'median'})
                    .sort_values(by='number', ascending = False)
                   )
type_number_data

,number
object_type,
столовая,103
ресторан,80
бар,35
буфет,32
кафе,30
кафетерий,6
предприятие быстрого обслуживания,5
закусочная,0
магазин (отдел кулинарии),0


В среднем самое большое число посадочных мест в столовых.  
Посмотрим на разброс значений по каждому типу заведений.

In [58]:
fig = px.box(rest_data, x="number", y="object_type", orientation='h', 
             labels={'number': 'число посадочных мест', 'object_type': ''})
fig.show()

Визуально больше всего выбросов у ресторанов. Есть даже ресторан на 1500 мест.  
Значения по кафетериям и закусочным выглядят практически идентично.  
Боксплоты по кафе и барам тоже похожи, но у кафе более длинный хвост до 533 мест.  
Также обнаружены бар на 1700 и столовая на 1400 посадочных мест.

In [59]:
# Напечатаем строчки с заведениями больше 920 мест
rest_data.query('number >= 920').sort_values(by='number', ascending=False)

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
12723,12723,199696,Arena by Soho Family,нет,бар,"город Москва, Кутузовский проспект, дом 36, ст...",1700,Кутузовский проспект
8148,8148,80291,Банкетный зал Шелк,нет,ресторан,"город Москва, Большой Саввинский переулок, дом...",1500,Большой Саввинский переулок
8345,8345,27750,СТОЛОВАЯ-БУФЕТ,нет,столовая,"город Москва, улица Волхонка, дом 15",1400,улица Волхонка
9064,9064,19719,КОМБИНАТ ПИТАНИЯ «УПРАВЛЕНИЕ АКАДЕМСЕРВИС»,нет,столовая,"город Москва, проспект Вернадского, дом 84, ст...",1288,проспект Вернадского
3686,3686,27026,РУМ СЕРВИС,нет,ресторан,"город Москва, площадь Европы, дом 2",1200,площадь Европы
4480,4480,27024,РУМ СЕРВИС,нет,ресторан,"город Москва, Кутузовский проспект, дом 2/1, с...",1200,Кутузовский проспект
2313,2313,26560,Ресторан «АЛЬФА»,нет,ресторан,"город Москва, Измайловское шоссе, дом 71, корп...",1040,Измайловское шоссе
9955,9955,171116,EATALY,нет,ресторан,"город Москва, Киевская улица, дом 2",920,Киевская улица


Бар на Кутузовском, банкетный зал, Комбинат питания, ресторан в гостинице, Eataly с ними все ясно.  
Рум сервис поисковики не обнаружили.

<font style="color:DeepPink; font-size:200%">•</font> Выделим в отдельный столбец информацию об улице из столбца address.

In [60]:
import re
def street_func_first(x):
    street = re.search(', [-\dА-яё№ ]+,', x)
    if street:
        return street.group()
    else:
        street = re.search('[-\dА-яё№ ]+,', x)
        if street:
            return street.group()
def street_func_second(x):
    street = re.search('[-\dА-яё№ ]+', x)
    if street:
        return street.group()
def street_func_third(x):
    street = re.search('[^-\s][-\dА-яё№ ]+', x)
    if street:
        return street.group()

In [61]:
rest_data['street'] = rest_data['address'].apply(lambda x: street_func_first(x))
rest_data['street'] = rest_data['street'].apply(lambda x: street_func_second(x))
rest_data['street'] = rest_data['street'].apply(lambda x: street_func_third(x))
rest_data.head()

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
0,0,151635,СМЕТАНА,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48,улица Егора Абакумова
1,1,77874,Родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35,улица Талалихина
2,2,24309,Кафе «Академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95,Абельмановская улица
3,3,21894,ПИЦЦЕТОРИЯ,да,кафе,"город Москва, Абрамцевская улица, дом 1",40,Абрамцевская улица
4,4,119365,Кафе «Вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50,Абрамцевская улица


<font style="color:DeepPink; font-size:200%">•</font> Построим график топ-10 улиц по количеству объектов общественного питания. Определим в каких районах Москвы находятся эти улицы?

In [62]:
# Определим ТОП10 улиц по количеству объектов
street_data = (rest_data.groupby('street')
             .agg({'id': 'count', 'number': 'median'})
             .sort_values(by='id', ascending=False)
             .reset_index()
            )
top10_street_data = street_data.head(10)
top10_street_data

,street,id,number
0,город Зеленоград,232,52.5
1,проспект Мира,204,46.5
2,Профсоюзная улица,182,24.0
3,Ленинградский проспект,171,40.0
4,Пресненская набережная,167,30.0
5,Варшавское шоссе,162,30.0
6,Ленинский проспект,148,45.0
7,поселение Сосенское,138,15.0
8,проспект Вернадского,132,40.0
9,Кутузовский проспект,114,40.0


На первом месте неожиданно оказался Зеленоград.  
Посмотрим, что там за адреса.

In [63]:
zelenograd_rest_data = rest_data[rest_data['address'].str.contains("город Зеленоград")].copy()
zelenograd_rest_data.sample(10)

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
13686,13686,211330,Пекарня «Дон Батон»,нет,кафе,"город Москва, город Зеленоград, корпус 1106",7,город Зеленоград
1405,1405,20199,ЭТО,нет,кафе,"город Москва, город Зеленоград, корпус 401",35,город Зеленоград
10843,10843,23378,ШКОЛА 1703,нет,столовая,"город Москва, город Зеленоград, Берёзовая алле...",100,город Зеленоград
4920,4920,84461,Столовая «ГБОУ Школа №852»,нет,столовая,"город Москва, город Зеленоград, корпус 1115",160,город Зеленоград
7211,7211,22324,АВТО-РЕКОРС,нет,кафе,"город Москва, город Зеленоград, Сосновая аллея...",20,город Зеленоград
7199,7199,23839,ШКОЛА 1806,нет,столовая,"город Москва, город Зеленоград, Берёзовая алле...",160,город Зеленоград
7228,7228,25936,Ресторан «Белл паб»,нет,ресторан,"город Москва, город Зеленоград, Яблоневая алле...",70,город Зеленоград
10533,10533,190332,Зефир,нет,кафе,"город Москва, город Зеленоград, корпус 618",35,город Зеленоград
2439,2439,69286,Кафе «Андерсон»,да,кафе,"город Москва, город Зеленоград, площадь Юности...",190,город Зеленоград
13914,13914,211636,Панорама,нет,ресторан,"город Москва, город Зеленоград, корпус 617А",40,город Зеленоград


В Зеленограде адреса столовых, буфетов в школах и кафе иногда указаны без улиц, вот в чем дело.  
В столбец street выведем улицу там, где она указана.

In [64]:
def zeleg_street_func_first(x):
    street = re.search(', [^город Зеленоград][-\dА-яё№ ]+,', x)
    if street:
        return street.group()
    else:
        street = re.search(', [-\dА-яё№ ]+,', x)
        if street:
            return street.group()
def zeleg_street_func_second(x):
    street = re.search('[-\dА-яё№ ]+', x)
    if street:
        return street.group()
def zeleg_street_func_third(x):
    street = re.search('[^-\s][-\dА-яё№ ]+', x)
    if street:
        return street.group()
    #zelenograd_rest_data = zelenograd_rest_data.drop('street', 1)
zelenograd_rest_data['street'] = zelenograd_rest_data['address'].apply(lambda x: zeleg_street_func_first(x))
zelenograd_rest_data['street'] = zelenograd_rest_data['street'].apply(lambda x: zeleg_street_func_second(x))
zelenograd_rest_data['street'] = zelenograd_rest_data['street'].apply(lambda x: zeleg_street_func_third(x))
zelenograd_rest_data.head()

,Unnamed: 0,id,object_name,chain,object_type,address,number,street
1386,1386,22331,СТОЛОВАЯ МИЭТ,нет,столовая,"город Москва, город Зеленоград, улица Юности, ...",56,улица Юности
1405,1405,20199,ЭТО,нет,кафе,"город Москва, город Зеленоград, корпус 401",35,город Зеленоград
1406,1406,68113,Френдс,нет,кафе,"город Москва, город Зеленоград, корпус 435",34,город Зеленоград
1407,1407,20105,Кафе «Граф Монте-Кристо»,нет,кафе,"город Москва, город Зеленоград, корпус 436",40,город Зеленоград
1408,1408,22982,Альфорно,нет,кафе,"город Москва, город Зеленоград, корпус 438",49,город Зеленоград


In [65]:
# Определим ТОП10 улиц по количеству объектов
zel_street_data = (zelenograd_rest_data.groupby('street')
             .agg({'id': 'count', 'number': 'median'})
             .sort_values(by='id', ascending=False)
             .reset_index()
            )
zel_street_data.head(10)

,street,id,number
0,город Зеленоград,120,75.5
1,Крюковская площадь,12,25.0
2,Панфиловский проспект,9,30.0
3,проспект Генерала Алексеева,7,45.0
4,площадь Юности,7,65.0
5,Сосновая аллея,6,14.5
6,Савёлкинский проезд,6,76.0
7,1-й Западный проезд,5,100.0
8,площадь Шокина,5,70.0
9,улица Юности,5,40.0


Там, где названия улиц не указаны, указаны только корпуса и они разбросаны по всему городу.  
Можно добавить информацию вручную, но данные по Зеленограду составляют 1,5% от анализируемых данных, не имеет смысла тратить на них много времени.  

Еще в ТОП-10 затесалось поселение Сосенское (в Новомосковском районе Москвы). Количество заведений там составляет меньше 1% от анализируемых данных.  

Для дальнейшего анализа мы их из выборки удалим.

In [66]:
street_data = street_data.drop(0, 0).reset_index(drop=True)
street_data = street_data.drop(6, 0).reset_index(drop=True)

Итак, вот наш ТОП-10 улиц по количеству объектов общественного питания:

In [67]:
top10_street_data = street_data.head(10).copy()
top10_street_data

,street,id,number
0,проспект Мира,204,46.5
1,Профсоюзная улица,182,24.0
2,Ленинградский проспект,171,40.0
3,Пресненская набережная,167,30.0
4,Варшавское шоссе,162,30.0
5,Ленинский проспект,148,45.0
6,проспект Вернадского,132,40.0
7,Кутузовский проспект,114,40.0
8,Каширское шоссе,112,25.0
9,Кировоградская улица,110,30.0


In [68]:
fig = px.bar(top10_street_data.sort_values(by='id', ascending=True), y='street', x='id', orientation='h', height=400,
             labels={'id': 'Количество объектов', 'street': ''}, range_x=(0,220),
             title='ТОП-10 улиц по количеству объектов общественного питания',
             template='plotly_white')
fig.update_xaxes(tickangle=45)
fig.show()

Добавим к ним названия районов, в которых расположены улицы.

In [70]:
# Импортируем в проект данные мосгаза
streets = pd.read_csv('datasets/mosgaz-streets.csv')
streets.head()

,streetname,areaid,okrug,area
0,Выставочный переулок,17,ЦАО,Пресненский район
1,улица Гашека,17,ЦАО,Пресненский район
2,Большая Никитская улица,17,ЦАО,Пресненский район
3,Глубокий переулок,17,ЦАО,Пресненский район
4,Большой Гнездниковский переулок,17,ЦАО,Пресненский район


In [71]:
top10_street_data = top10_street_data.merge(streets, left_on='street', right_on='streetname', how='left')
top10_street_data

,street,id,number,streetname,areaid,okrug,area
0,проспект Мира,204,46.5,проспект Мира,70,СВАО,Алексеевский район
1,проспект Мира,204,46.5,проспект Мира,86,СВАО,Ярославский Район
2,проспект Мира,204,46.5,проспект Мира,78,СВАО,Район Марьина роща
3,проспект Мира,204,46.5,проспект Мира,79,СВАО,Останкинский район
4,проспект Мира,204,46.5,проспект Мира,81,СВАО,Район Ростокино
5,проспект Мира,204,46.5,проспект Мира,82,СВАО,Район Свиблово
6,проспект Мира,204,46.5,проспект Мира,16,ЦАО,Мещанский район
7,Профсоюзная улица,182,24.0,Профсоюзная улица,123,ЮЗАО,Академический район
8,Профсоюзная улица,182,24.0,Профсоюзная улица,132,ЮЗАО,Район Черемушки
9,Профсоюзная улица,182,24.0,Профсоюзная улица,134,ЮЗАО,Район Ясенево


Крупные улицы в Москве часто пересекают сразу несколько районов. Без учета номеров домов сопоставить заведения с районами города не получится.  
Но можно более обощенно прикинуть округа.

In [72]:
okrug_top10 = top10_street_data.groupby('okrug').agg({'street':'count'}).sort_values(by='street',ascending=False).reset_index()
okrug_top10['col_%'] = round(okrug_top10['street'] / okrug_top10['street'].sum() *100,2)
okrug_top10

,okrug,street,col_%
0,ЮАО,14,29.17
1,ЮЗАО,14,29.17
2,ЗАО,7,14.58
3,СВАО,6,12.50
4,САО,4,8.33
5,ЦАО,3,6.25


Итак, большинство улиц из ТОП-10 относятся к югу города:
* ЮАО 30%
* ЮЗАО 30%

<font style="color:DeepPink; font-size:200%">•</font> Найдем число улиц с одним объектом общественного питания. Определим в каких районах Москвы находятся эти улицы?

In [73]:
one_obj_street_data = street_data.query('id == 1')

In [74]:
print('Улиц с одним объектом общественного питания: ',len(one_obj_street_data))

Улиц с одним объектом общественного питания:  562


Добавим к ним данные по районам и округам.

In [75]:
one_obj_street_data = one_obj_street_data.merge(streets, left_on='street', right_on='streetname', how='left')

In [76]:
one_obj_street_data.head(10)

,street,id,number,streetname,areaid,okrug,area
0,56-й километр Московской Кольцевой Автодороги,1,1.0,NaN,NaN,NaN,NaN
1,улица Дубки,1,35.0,улица Дубки,67.0,САО,Тимирязевский Район
2,7-я улица Лазенки,1,200.0,7-я улица Лазенки,42.0,ЗАО,Район Ново-Переделкино
3,Бабаевская улица,1,90.0,Бабаевская улица,26.0,ВАО,Район Сокольники
4,улица Достоевского,1,60.0,улица Достоевского,19.0,ЦАО,Тверской район
5,улица Достоевского,1,60.0,улица Достоевского,78.0,СВАО,Район Марьина роща
6,улица Достоевского,1,60.0,улица Достоевского,16.0,ЦАО,Мещанский район
7,улица Капотня,1,220.0,NaN,NaN,NaN,NaN
8,улица Жуковского,1,50.0,улица Жуковского,13.0,ЦАО,Басманный район
9,Авиационный переулок,1,62.0,Авиационный переулок,54.0,САО,Район Аэропорт


По некоторым улицам район не определился.  
Например улица Капотня. Посмотрим, какие данные с упоминанием Капотни есть в базе по районам.

In [77]:
streets[streets['streetname'].str.contains("Капотня")]

,streetname,areaid,okrug,area
3449,1-й квартал Капотня,112,ЮВАО,Район Капотня
3450,2-й квартал Капотня,112,ЮВАО,Район Капотня
3451,3-й квартал Капотня,112,ЮВАО,Район Капотня
3452,4-й квартал Капотня,112,ЮВАО,Район Капотня
3453,5-й квартал Капотня,112,ЮВАО,Район Капотня


В базе есть кварталы, а улицы нет. Хотя на самом деле такая улица существует и относится к ЮВАО.  
Выходит данные в базе не полные.

Посмотрим, по скольким улицам не определяется район.

In [78]:
print('Не получилось определить район по {} улицам.'.format(len(one_obj_street_data[one_obj_street_data['area'].isnull()])))

Не получилось определить район по 46 улицам.


Можно заполнить пропуски вручную, но это займет много времени. А данных нет по 46 заведениям, это немного. Информацией по ним можно пренебречь.  
Удалим их из выборки.

In [79]:
# Удаляем строки с пропущенными значениями в районе города
one_obj_street_data = one_obj_street_data.dropna(subset=['area'])

Посмотрим, в каких округах расположены улицы с заведениями с одним посадочным местом.

In [80]:
okrug_one_obj = (one_obj_street_data
                 .groupby('okrug')
                 .agg({'street':'count'})
                 .sort_values(by='street', ascending=False)
                 .reset_index()
                )
okrug_one_obj['col_%'] = round(okrug_one_obj['street'] / okrug_one_obj['street'].sum() *100,2)
okrug_one_obj

,okrug,street,col_%
0,ЦАО,184,32.74
1,ВАО,72,12.81
2,СВАО,66,11.74
3,САО,55,9.79
4,ЮВАО,55,9.79
5,ЗАО,43,7.65
6,ЮАО,30,5.34
7,ЮЗАО,28,4.98
8,СЗАО,27,4.80
9,Зеленоград,2,0.36


* 33% заведений с одним посадочным местом расположено в ЦАО.
* 13% в ВАО
* 12% в СВАО

<font style="color:DeepPink; font-size:200%">•</font> Посмотрим на распределение количества посадочных мест для улиц с большим количеством объектов общественного питания. Найдем закономерности.

In [81]:
# Сохраним названия заведений из ТОП10 в список
top10_street_names = top10_street_data['street'].unique()

# Отфильтруем таблицу с заведениями по названиям из списка ТОП10
filt_df = rest_data['street'].isin(top10_street_names)
filt_street_data_top10 = rest_data[filt_df]

# Отдельно добавим столбик с общим количеством заведений на этой улице и отсортируем в порядке убывания количества точек
quantity_of_street = top10_street_data[['street','id']]
quantity_of_street.columns = ['street', 'quantity_of_street']
filt_street_data_top10 = filt_street_data_top10.merge(quantity_of_street, on='street').sort_values(by='quantity_of_street')

In [82]:
fig = px.box(filt_street_data_top10, x="number", y="street", orientation='h', 
             labels={'number': 'число посадочных мест', 'street': ''})
fig.show()

Верхняя граница распределения посадочных мест для заведений с большим количеством объектов общественного питания от 100 до 173 посадочных мест.

<font style="color:Blue; font-size:400%">•</font> **Вывод**

В мировой практике заведения с роботами, обслуживающими посетителей, существуют разных форматов: от небольших заведений (до 30 посадочных мест) в отдаленных районах города, до обширной зоны фудкортов.  
Роботы перемещаются по залу, убирают пустую посуду, готовят, приносят новые блюда, танцуют на входе, завлекая новых посетителей, умеют произносить стандартный набор фраз, во время ожидания новых заказов роботы жонглируют тарелками, дерутся за кухонную утварь, устраивают комические словесные поединки.

Для открытия первого заведения стоит рассматривать улицы с большой проходимостью, на которых расположено много объектов общественного питания или фудкорты.  
Оригинальный формат скорее всего привлечет посетителей других заведений своей новизной и экзотикой.

Для выбора конкретного района стоит провести дополнительное исследование. Исходя из наших данных, можно порекомендовать ориентироваться на ЮАО и ЮЗАО.

Самый распространенный тип как среди сетевых, так и не сетевых заведений - кафе.  
На проходных улицах стоит ограничиться залом до 60 посадочных мест.

После анализа успешных крупных сетей, можно сделать вывод, что количество мест в заведении должно расти пропорционально количеству новых точек продаж в сети.

Презентация <a href='https://drive.google.com/open?id=1wdbL4bcnBDxCsu8YG4jmwMmL8FukxJdw'>на Google.drive</a>